In [18]:
from diffusers import DiffusionPipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
model_id = 'black-forest-labs/FLUX.1-dev'
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.bfloat16).to('cuda')

In [9]:
loras_paths_dict = {
    'base': None,
    '2000': '/root/output/trained_model_a/trained_model_a_000002000.safetensors',
    '5000': '/root/output/trained_model_a/trained_model_a_000005000.safetensors',
    '10000': '/root/output/trained_model_a/trained_model_a_000010000.safetensors',
    '15000': '/root/output/trained_model_a/trained_model_a_000015000.safetensors',
    '25000': '/root/output/trained_model_a/trained_model_a_000025000.safetensors',

}

In [51]:
trigger_word = 'pic_a, luxurious style, gold, high importance, ultra close up'

In [29]:
import matplotlib.pyplot as plt

def plot_images(images: dict[str, 'Image.Image']):
    fig, axes = plt.subplots(1, len(images), figsize=(len(images) * 4, 6), facecolor='black')  # darker background
    for i, (name, img) in enumerate(images.items()):
        axes[i].imshow(img)
        axes[i].set_title(name, color='white')  # white titles for contrast
        axes[i].axis('off')  # hide axis
        axes[i].set_facecolor('black')  # set image panel background
    plt.tight_layout()
    plt.show()

In [54]:
def inference_with_lora(prompt: str, lora_path: str | None = None):
    pipe.unload_lora_weights()
    if lora_path:
        pipe.load_lora_weights(lora_path, adapter_name="high")
        pipe.set_adapters(["high"], adapter_weights=[0.9])
    prompt = f'{trigger_word}, {prompt}'
    image = pipe(prompt=prompt, height=1024, width=1024, num_inference_steps=20, guidance_scale=3.0).images[0]
    if lora_path:
        pipe.unload_lora_weights()
    return image



In [55]:
def multi_inference(prompt: str, lora_paths: dict[str, str]):
    images = {}
    for name, lora_path in lora_paths.items():
        images[name] = inference_with_lora(prompt, lora_path)
    return images

In [ ]:
prompt = 'big size, ultra close up portrait, pica style, luxurious style, gold, high importance, warm tones, slot icon style, ultra close up, Puss in Boots wearing oversized golden hat with ruby feather, ornate scarlet cloak with embroidered lions, holding a glowing rapier encrusted with emeralds, background: tavern fireplace and flickering candlelight — dashing, dangerous, deluxe'

imgs = multi_inference(prompt, loras_paths_dict)

In [ ]:
plot_images(imgs)

In [ ]:
prompt = "pica style, big size, ultra close up portrait, takes up all the space in the image, luxurious style, gold, high importance, slot icon style, warm tones, Golden Elvis-style performer mid-spin with glowing red guitar, star-shaped sunglasses, cape trailing fire and glitter, background: Vegas stage bursting with light and gold coins'ArithmeticErro"
imgs = multi_inference(prompt, loras_paths_dict)
plot_images(imgs)


In [ ]:
plot_images(imgs)

In [46]:
icon_prompts = [
    "A majestic depiction of Harry Potter himself, clad in robes, his iconic round glasses shimmering with radiant glimmers, surrounded by swirling sparks of magic, set against an background of enchanted castle towers.",
    "Futuristic robot head icon, chrome and blue color scheme, 3D style",
    # "Vintage camera icon, hand-drawn sketch style, sepia tones",
    # "Magic potion bottle icon with glowing liquid and sparkles, fantasy theme",
    # "Cyberpunk eye scanner icon, neon lights and glitch effect",
    # "Abstract leaf icon made from geometric shapes, eco-friendly theme",
    # "Retro game controller icon, pixel art style, colorful buttons",
    # "Crystal gemstone icon with light reflections, RPG inventory style",
    # "Cute cat face icon with big eyes and pastel colors, kawaii style",
    # "Mystic rune icon engraved in stone, ancient magical feel",
    # "Space rocket icon launching with flames and stars, cartoon style",
    # "Chef hat and spoon icon, minimal design for cooking app",
    # "Treasure chest icon slightly open with gold coins inside, game style",
    # "Wi-Fi signal icon made of clouds, sky background, soft aesthetic",
    # "Time travel portal icon, spiraling clock and vortex colors",
    # "Notebook and pen icon, flat style, suitable for productivity app",
    # "Alien face icon with big black eyes, neon green, sci-fi theme",
    # "Digital brain icon made of circuit lines, AI and tech style",
    # "Ice cream cone icon melting slightly, colorful and summer vibe",
    # "Witch hat icon with stars and moon symbols, dark cartoon style"
]


In [ ]:
list(loras_paths_dict.values())

In [48]:
def inference_with_multiple_prompts(prompts: list[str]):
    images = {}
    for prompt in prompts:
        images[prompt] = multi_inference(prompt, loras_paths_dict)
    return images



In [ ]:
len(icon_prompts)

In [ ]:
res = inference_with_multiple_prompts(icon_prompts)

In [ ]:
for prompt, images in res.items():
    plot_images(images)